In [ ]:
import numpy as np
import avapi
import avstack

In [ ]:
nusc_data_dir = '../../data/nuScenes'
NSM = avapi.nuscenes.nuScenesManager(nusc_data_dir)
NSD = NSM.get_scene_dataset_by_name('scene-0103')

In [ ]:
img = NSD.get_image(1, 'main_camera')
img.view()

## Test Radar Tracking

In [ ]:
nusc_data_dir = '../../data/nuScenes'
NSM = avapi.nuscenes.nuScenesManager(nusc_data_dir)
NSD = NSM.get_scene_dataset_by_name('scene-0103')

In [ ]:
last_frame = 20
tracker = avstack.modules.tracking.tracker3d.BasicRazelTracker()
for frame in NSD.frames[:last_frame]:
    ts = NSD.get_timestamp(frame)
    radar = NSD.get_radar(frame)
    objects = NSD.get_objects(frame)
    img = NSD.get_image(frame, "main_camera")
    detections = [avstack.modules.perception.detections.RazelDetection(
                    radar.source_identifier, radar.data[i,:3]) for i in range(radar.data.shape[0])]
    tracks = tracker(t=ts, frame=frame, detections_nd=detections)
    avapi.visualize.snapshot.show_image_with_boxes(img, tracks, inline=True)

In [ ]:
frame = 6
pc = NSD.get_lidar(frame=frame)
pc.data = np.zeros((0,4))
objects = NSD.get_objects(frame=frame)
rad = NSD.get_radar(frame=frame)
det_xyz = avstack.geometry.transformations.matrix_spherical_to_cartesian(rad.data)

# -- vectors from detections
vectors = []
for row in det_xyz:
    head = row[:3]
    tail = row[:3] + row[:3]/np.linalg.norm(row[:3]) * row[3]
    vectors.append(avstack.geometry.primitives.VectorHeadTail(head=head, tail=tail, origin=rad.calibration.origin))
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

# -- vectors from tracks
vectors = []
for track in tracks:
    head = track.x[:3]
    tail = track.x[:3] + track.x[3:6]
    vectors.append(avstack.geometry.primitives.VectorHeadTail(head=head, tail=tail, origin=rad.calibration.origin))
avapi.visualize.snapshot.show_lidar_bev_with_boxes(pc, boxes=objects, vectors=vectors)

In [ ]:
# put data into common format
frame = last_frame
rad = NSD.get_radar(frame=frame)
det_xyz = avstack.geometry.transformations.matrix_spherical_to_cartesian(rad.data)
trk_xyz = np.array([track.x[:3] for track in tracks])
objects = NSD.get_objects(frame=frame)

# verify each object has close radar for sanity check
for obj in objects:
    obj.change_origin(rad.origin)
    min_det_dist = min(np.linalg.norm(obj.position.vector[:2] - det_xyz[:,:2], axis=1))
    min_trk_dist = min(np.linalg.norm(obj.position.vector[:2] - trk_xyz[:,:2], axis=1))
    print(f'Minimum distance from object to closest det is {min_det_dist:4.4f}')
    print(f'Minimum distance from object to closest trk is {min_trk_dist:4.4f}\n')
          